In [28]:
%pip install scikit-learn


   ---------------------------------------- 10.6/10.6 MB 851.3 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline #hugging face pipeline
from catboost import CatBoostClassifier

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import ast
import os



In [18]:
def createDataSet(data, reviews):
    merged_df = pd.merge(data[['business_id', 'review_count', 'attributes', 'is_open']], reviews,on='business_id')
    attributes_df = pd.json_normalize(merged_df['attributes'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {}))

    merged_df['label_divided'] = merged_df['label'] / merged_df['review_count']
    data = merged_df[['review_count', 'label_divided']]
    data['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
    return data

In [17]:

x_train = pd.read_csv('./kaggle/input/desafioutfpr2024/X_trainToronto.csv')
x_scores = pd.read_csv('exported_review_sum.csv')
y_train = (pd.merge(x_train[['business_id','destaque']], x_scores,on='business_id'))['destaque']


createDataSet(x_train, x_scores)



C:\Users\danie\AppData\Local\Temp\ipykernel_19912\2027405084.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = attributes_df['RestaurantsPriceRange2'].replace('None', np.nan).astype(float)
C:\Users\danie\AppData\Local\Temp\ipykernel_19912\2027405084.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
C:\Users\danie\AppData\Local\Temp\ipykernel_19912\2027405084.py:10: FutureWarning: A value is trying to 

,review_count,label_divided,is_open,price,attributes
0,99,0.449040,1,3.000000,9
1,3,1.331739,1,2.000000,2
2,18,0.221720,1,1.948135,2
3,3,0.338114,0,1.948135,1
4,3,-0.999251,1,3.000000,0
...,...,...,...,...,...
17577,16,-0.251374,1,1.000000,12
17578,4,0.998909,1,2.000000,2
17579,8,-0.125223,1,2.000000,1
17580,9,-0.776320,1,2.000000,9


In [12]:

x_test = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')
x_test_scores = pd.read_csv('exported_review_sum_test.csv')

createDataSet(x_test, x_test_scores)



C:\Users\danie\AppData\Local\Temp\ipykernel_19912\2027405084.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = attributes_df['RestaurantsPriceRange2'].replace('None', np.nan).astype(float)
C:\Users\danie\AppData\Local\Temp\ipykernel_19912\2027405084.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['attributes'] = attributes_df.loc[:, attributes_df.eq('True').any()].count(axis=1)
C:\Users\danie\AppData\Local\Temp\ipykernel_19912\2027405084.py:10: FutureWarning: A value is trying to 

,review_count,label_divided,is_open,price,attributes
0,9,-0.110696,1,2.000000,10
1,6,-0.000291,1,1.938644,0
2,8,0.998930,1,3.000000,0
3,13,0.844799,0,1.938644,3
4,3,0.333033,0,3.000000,0
...,...,...,...,...,...
1319,4,-0.000277,0,2.000000,1
1320,3,0.998871,1,4.000000,0
1321,62,0.788955,1,1.938644,1
1322,9,0.554772,1,1.000000,6


In [7]:
from catboost import Pool
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
eval_data = Pool(x_train, y_train)
x_train


array([[ 1.19056544e+00,  1.90711216e-01,  5.64870517e-01,
         1.79193845e+00,  1.23769239e+00],
       [-4.00175094e-01,  1.85402857e+00,  5.64870517e-01,
         8.83569439e-02, -5.33581150e-01],
       [-1.51621886e-01, -2.37640675e-01,  5.64870517e-01,
        -1.04674545e-16, -5.33581150e-01],
       ...,
       [-3.17324025e-01, -8.91404663e-01,  5.64870517e-01,
         8.83569439e-02, -7.86620226e-01],
       [-3.00753811e-01, -2.11830231e+00,  5.64870517e-01,
         8.83569439e-02,  1.23769239e+00],
       [-3.67034666e-01,  4.73757858e-01,  5.64870517e-01,
        -1.04674545e-16, -7.86620226e-01]])

In [6]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV


# Create a CatBoost classifier object
classifier = CatBoostClassifier(iterations=1000,
                            per_float_feature_quantization=['1:border_count=1024'],
                            learning_rate=0.1,
                            depth=5,
                            loss_function='CrossEntropy',
                           task_type="GPU",
                           eval_metric='F1',
                           devices='0')

# Train the classifier on x_train and y_train
classifier.fit(x_train, y_train, eval_set = eval_data)





0:	learn: 0.7679714	test: 0.7679714	best: 0.7679714 (0)	total: 134ms	remaining: 2m 13s
1:	learn: 0.7737705	test: 0.7737705	best: 0.7737705 (1)	total: 243ms	remaining: 2m 1s
2:	learn: 0.7738307	test: 0.7738307	best: 0.7738307 (2)	total: 340ms	remaining: 1m 53s
3:	learn: 0.7730331	test: 0.7730331	best: 0.7738307 (2)	total: 447ms	remaining: 1m 51s
4:	learn: 0.7717487	test: 0.7717487	best: 0.7738307 (2)	total: 540ms	remaining: 1m 47s
5:	learn: 0.7735036	test: 0.7735036	best: 0.7738307 (2)	total: 626ms	remaining: 1m 43s
6:	learn: 0.7774175	test: 0.7774175	best: 0.7774175 (6)	total: 711ms	remaining: 1m 40s
7:	learn: 0.7780077	test: 0.7780077	best: 0.7780077 (7)	total: 795ms	remaining: 1m 38s
8:	learn: 0.7793705	test: 0.7793705	best: 0.7793705 (8)	total: 899ms	remaining: 1m 38s
9:	learn: 0.7792859	test: 0.7792859	best: 0.7793705 (8)	total: 1.01s	remaining: 1m 39s
10:	learn: 0.7822289	test: 0.7822289	best: 0.7822289 (10)	total: 1.1s	remaining: 1m 38s
11:	learn: 0.7827399	test: 0.7827399	best: 

In [ ]:
# Define the parameter grid
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.1, 0.3, 0.5],
    'depth': [5, 10, 15],
    'loss_function': ['Logloss', 'CrossEntropy']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5)

# Fit the grid search to the training data
grid_search.fit(x_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [52]:
# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'depth': 5, 'iterations': 200, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
Best Score: 0.9185531805050264


In [8]:
classifier.get_feature_importance()

array([26.02716985, 60.27612978,  1.88846467,  1.42201907, 10.38621663])

In [37]:
x_submission = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')

# Create a new DataFrame with business_id and predictions
predictions = pd.DataFrame({'business_id': x_submission['business_id'], 'destaque': grid_search.best_estimator_.predict(x_test)})

# Export the DataFrame to a CSV file
predictions.to_csv('predictions6.csv', index=False)
